In [17]:
#!pip install umap-learn
!pip install umap-learn

In [125]:
import os
os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
import pickle
import numpy as np
import umap
import matplotlib.pyplot as plt
import umap.umap_ as umap

In [23]:
!pip show umap

In [135]:
filename = 'glove_6B/glove2word2vec_model.sav'
with open(filename, 'rb') as file:
    model = pickle.load(file)

# Extract word vectors from the model
word_vectors = model

In [136]:
words_of_interest = ["beach", "ocean", "sun", "sand", "waves"]

# Remove missing words and duplicates from the list
unique_words_of_interest = list(set(words_of_interest).intersection(model.key_to_index))

# Retrieve the word vectors for the selected words
vectors_of_interest = [model[word] for word in words_of_interest if word in model.key_to_index]

if not vectors_of_interest:
    print("No valid word vectors found. Please check your input data.")
else:
    reducer = umap.UMAP(n_neighbors=100)
    umap_embeddings = reducer.fit_transform(vectors_of_interest)

/Users/AprilCoffee/opt/anaconda3/envs/tf_cpu/lib/python3.7/site-packages/umap/umap_.py:2434: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  "n_neighbors is larger than the dataset size; truncating to "


In [137]:

def drawPlt(umap_embeddings, words_of_interest, highlight_words=None):
    # Create a scatter plot of the UMAP embeddings
    plt.figure(figsize=(10, 8))
    
    # Initialize colors for all points as blue
    point_colors = ['blue'] * len(words_of_interest)
    
    # Change the color for highlighted words to red
    if highlight_words:
        for i, word in enumerate(words_of_interest):
            if word in highlight_words:
                point_colors[i] = 'red'
    
    # Scatter plot all words with smaller circles and updated colors
    plt.scatter(umap_embeddings[:, 0], umap_embeddings[:, 1], marker='.', s=20, c=point_colors, alpha=0.5)
    
    # Annotate points with words
    for i, word in enumerate(words_of_interest):
        if i < umap_embeddings.shape[0]:  # Check if the index is within bounds
            plt.annotate(word, (umap_embeddings[i, 0], umap_embeddings[i, 1]), fontsize=10)
    
    plt.title("UMAP Visualization of GloVe Word Embeddings")
    plt.xlabel("")
    plt.ylabel("")
    plt.show()


In [138]:
from sklearn.metrics.pairwise import cosine_similarity

# Choose a target word
target_word = "ocean"
words_of_interest = []
words_of_interest.append(target_word)
# Ensure the target word is in the vocabulary
if target_word in model.key_to_index:
    # Get the vector for the target word
    target_vector = model[target_word].reshape(1, -1)

    # Calculate cosine similarities between the target word and all other words
    similarities = cosine_similarity(target_vector, model.vectors)

    # Find the top N similar words (excluding the target word itself)
    top_n = 200  # Adjust as needed
    similar_word_indices = similarities.argsort()[0][-top_n - 1:-1][::-1]

    # Get the similar words and their similarity scores
    similar_words = [model.index_to_key[i] for i in similar_word_indices]
    similarity_scores = [similarities[0][i] for i in similar_word_indices]
    words_of_interest.extend(similar_words)

    # Print the similar words and their similarity scores
    #for word, score in zip(similar_words, similarity_scores):
    #    print(f"{word}: {score:.4f}")
#else:
#    print(f"{target_word} is not in the vocabulary.")


In [139]:
vectors_of_interest = [model[word] for word in words_of_interest if word in model.key_to_index]

if not vectors_of_interest:
    print("No valid word vectors found. Please check your input data.")
else:
    reducer = umap.UMAP(n_neighbors=100)
    umap_embeddings = reducer.fit_transform(vectors_of_interest)

In [140]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

# Assuming you have loaded GloVe embeddings and vectors_of_interest
# vectors_of_interest is a list of word vectors

# Number of clusters (you can adjust this)
num_clusters = 20

# Normalize the word vectors (important for K-Means)
normalized_vectors = normalize(vectors_of_interest)

# Initialize and fit K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(normalized_vectors)

# Get the cluster labels for each word vector
cluster_labels = kmeans.labels_

# Print the words and their respective clusters
#for word, cluster in zip(words_of_interest, cluster_labels):
#    print(f"{word}: Cluster {cluster}")

# Analyze and visualize the clusters as needed


In [141]:
import matplotlib.pyplot as plt
import umap
import numpy as np

def drawPlt(umap_embeddings, words_of_interest, target_word, cluster_labels, focus_cluster=None, opacity=0.5, label_opacity=1.0, label_size=12):
    # Create a scatter plot of UMAP embeddings
    plt.figure(figsize=(12, 8))

    # Define unique colors for each cluster
    unique_clusters = set(cluster_labels)
    colors = plt.cm.viridis(np.linspace(0, 1, len(unique_clusters)))

    # Lists to store cluster labels
    cluster_label_lists = [[] for _ in range(len(unique_clusters))]

    # Plot points in focus cluster with full opacity
    for cluster in unique_clusters:
        mask = cluster_labels == cluster
        if cluster == focus_cluster:
            plt.scatter(
                umap_embeddings[mask, 0], umap_embeddings[mask, 1],
                label=f"Cluster {cluster}", color=colors[cluster], alpha=1.0
            )
            label_alpha = 1.0  # Full opacity for labels of the focus cluster
            label_size_cluster = label_size  # Adjust label size for the focus cluster
        else:
            plt.scatter(
                umap_embeddings[mask, 0], umap_embeddings[mask, 1],
                color=colors[cluster], alpha=opacity
            )
            label_alpha = label_opacity  # Adjusted opacity for labels of other clusters
            label_size_cluster = 8  # Smaller label size for other clusters

        # Annotate points with words and set label opacity and size
        for i, word in enumerate(words_of_interest):
            if cluster_labels[i] == cluster:
                plt.annotate(word, (umap_embeddings[i, 0], umap_embeddings[i, 1]), alpha=label_alpha, fontsize=label_size_cluster)
                cluster_label_lists[cluster].append(word)

    plt.title("UMAP Visualization of GloVe Word Embeddings")
    #plt.xlabel("UMAP Dimension 1")
    #plt.ylabel("UMAP Dimension 2")
    plt.legend()

    
    image_folder = "cluster_images"
    os.makedirs(image_folder, exist_ok=True)
    plt.savefig(f"{image_folder}/cluster_visualization_{focus_cluster}.png", dpi=300, bbox_inches='tight', format='png')
  
    # Show the plot
    
    # Optionally, you can specify the DPI (dots per inch) for resolution, and the format (e.g., PNG, JPEG, SVG)
    #plt.savefig('my_high_res_plot.png', dpi=300, format='png')

    plt.show()

    # Return the cluster label lists
    return cluster_label_lists

# Prepare to save label data for CSV
all_labels = []

# Generate and save plots
for i in range(20):
    cluster_label_lists = drawPlt(umap_embeddings, words_of_interest, target_word, cluster_labels, focus_cluster=i, opacity=0.3, label_opacity=0.5, label_size=14)
    
    # Collect label data
    for cluster, labels in enumerate(cluster_label_lists):
        for label in labels:
            all_labels.append([cluster, label])


NameError: name 'ax' is not defined

<Figure size 864x576 with 0 Axes>

In [ ]:
cluster_label_lists[0]

In [ ]:

import openai
import api_key
# Setting up enviornment
from base64 import b64decode
from pathlib import Path
import os
import io
import warnings
from PIL import Image
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation


os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'
os.environ['STABILITY_KEY'] = api_key.dreamStudio_key()


stability_api = client.StabilityInference(
    key=os.environ['STABILITY_KEY'], # API Key reference.
    verbose=True, # Print debug messages.
    engine="stable-diffusion-768-v2-1", # Set the engine to use for generation. 
)


def chatGPT_ads(input_text):

    openai.api_key = api_key.openai_key()
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 1.2,
            messages=[
                {"role": "system", "content": 
                '''
                you are a travel agency, that has the ability to write fantastic advertisements based on some fake locations

                I want you to write me SHORT describtion, give briefly. based on the keyword given.
                
               '''
                },

                #{"role": "user", "content": " A photorealistic, Spectacular nature landscape, highly detailed, photo-postcard of the coast, including mountain, landscape, river, waterfall "},
                #{"role": "assistant", "content":"Spectacular nature landscape, background mountains, river with waterfall, sharp focus, hyper detailed, detail colors, haze, fog, pine trees, depth, soft contrast,  golden hour colors, photo realistic, ultra realistic, cinematic landscape photography, cinematic ligthining"},
                {"role": "user", "content": 
             '''IDEA: short describtion based on the following text''' + input_text +"NO CANVAS"}
            ]
    )
    #print(response['choices'][0]['message']['content'])
    output = response['choices'][0]['message']['content']
    return output
    #os.system("say -v anna '"+output+"'") 
    #os.system("say '"+output+"'") 

def chatGPT(input_text):

    openai.api_key = api_key.openai_key()

    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature = 1.2,
            messages=[
                {"role": "system", "content": 
                '''
                you are a stable diffusion prompt generating machine.

                Stable Diffusion is an AI art generation model similar to DALLE.
                Below is a list of prompts that can be used to generate images with Stable Diffusion:

                - design, interrior, interriordesign Masterpiece, (photorealistic:1.4), the best quality, ultra-high res, photorealistic, realistic detailed, poster photograph, sharp focus, realistic landscape photo, shot with iPhone 13 Pro Max, shot with nikon z9 and nikkor z 70-200mm f/2. 8 VR S lens, waterall in the distance, a waterfall in the fall with a bright sky, sunset panorama, colorful digital screens) xf iq4, the glimmering orange dawn, realistic, photorealistic, aerial, drone shot, nature, ambient light, deer, detailed, hyperrealistic, sharp details, lens flare, beauty of nature, best quality, (golden hour lighting), muted colors,16K, 8K, 4K, flora and fauna, the essence of fall, colorful leaves, realistic water, realistic trees, realistic landscape, october, leaves on ground, raining, sunny, A picturesque view of a tranquil forest with leaves transitioning from the lush greens of summer to the golden hues of fall. The sun's rays pierce through the canopy, creating a play of light and shadow, emphasizing the crimson and amber tones, capturing the magic of the changing seasons, japan, tokyo, street, temple, temple gate ,#Portraiture Style-3 + Anamorphic + epic composition + golden-ratio + HDR10 + cinematography + #aweinspiring –v5 –q 2 –c 20” --style raw, HDR, ray tracing, highly detailed, 8k, ((fog)), dramatic, detailed shadows, fantastic realism, volumetric light, hyper detailed, ArtStation hq, (square aspect ratio), <lora:add_detail:0.8>, <lora:Fnaf_Vanny:0.2>
                - Spectacular nature landscape, background mountains, river with waterfall, sharp focus, hyper detailed, detail colors, haze, fog, pine trees, depth, soft contrast,  golden hour colors, photo realistic, ultra realistic, cinematic landscape photography, cinematic ligthining
                - ghost inside a hunted room, art by lois van baarle and loish and ross tran and rossdraws and sam yang and samdoesarts and artgerm, digital art, highly detailed, intricate, sharp focus, Trending on Artstation HQ, deviantart, unreal engine 5, 4K UHD image
                - red dead redemption 2, cinematic view, epic sky, detailed, concept art, low angle, high detail, warm lighting, volumetric, godrays, vivid, beautiful, trending on artstation, by jordan grimmer, huge scene, grass, art greg rutkowski
                -The Legend of Zelda landscape  atmospheric, hyper realistic, 8k, epic composition, cinematic, octane render, artstation landscape vista photography by Carr Clifton & Galen Rowell, 16K resolution, Landscape veduta photo by Dustin Lefevre & tdraw, 8k resolution, detailed landscape painting by Ivan Shishkin, DeviantArt, Flickr, rendered  in Enscape, Miyazaki, Nausicaa Ghibli, Breath of The Wild, 4k detailed post processing, artstation, rendering by octane, unreal engine —ar 16:9
                -The Lord of The Rings landscape  atmospheric, hyper realistic fantasy, 8k, epic composition, cinematic, octane render, highly detailed,artstation landscape vista painted by Carr Clifton & Galen Rowell, 16K resolution, Landscape veduta photo by Dustin Lefevre & tdraw, 8k resolution, detailed landscape painting by Ivan Shishkin, DeviantArt, Flickr, rendered  in Enscape, Miyazaki, Nausicaa Ghibli, Breath of The Wild, 4k detailed post processing,acrilic mode, artstation, rendering by octane, unreal engine —ar 16:9
                -beautiful crystal geode in a cavern, landscape, alex ross, eddie mendoza, raphael lacoste, sebastian ludke, concept art, matte painting, highly detailed, rule of thirds, dynamic lighting, cinematic, realism, realistic, photo real, detailed, magnificiant landscape, denoised, centerd 
                - ultra realistic illustration of steve urkle as the hulk, intricate, elegant, highly detailed, digital painting, artstation, concept art, smooth, sharp focus, illustration, art by artgerm and greg rutkowski and alphonse mucha
                -Landscape photo ultra realistic half bullfrog and half old hag woman with 4 arms in a swamp, dark fantasy, dead trees protrude from water dark castle in the far background, dramatic light, pale sunrise, cinematic lighting, battered, low angle, trending on artstation, 4k, hyper realistic, focused, extreme details, unreal engine 5, cinematic, masterpiece, art by studio ghibli, intricate artwork by john william turner
                – alberto seveso and geo2099 style, A highly detailed and hyper realistic portrait of a gorgeous young ana de armas, lisa frank, trending on artstation, butterflies, floral, sharp focus, studio photo, intricate details, highly detailed, by Tvera and wlop and artgerm


                taking this as example
                "Spectacular nature landscape, background mountains, river with waterfall, sharp focus, hyper detailed, detail colors, haze, fog, pine trees, depth, soft contrast,  golden hour colors, photo realistic, ultra realistic, cinematic landscape photography, cinematic ligthining"

                short description of the scene is :Spectacular nature landscape, background mountains, river with waterfall" 
                and followed by modifiers: sharp focus, hyper detailed, detail colors, haze, fog, pine trees, depth, soft contrast,  golden hour colors, photo realistic, ultra realistic, cinematic landscape photography, cinematic ligthining


                I want you to write me a prompt exactly about the idea written after IDEA. 
                Following the structure of the example prompts. 
                This means a VERY SHORT description of the scene, 
                and then followed by SHORT modifiers of single words divided by commas to alter the mood, style, lighting, and more.

               '''
                },

                {"role": "user", "content": " A photorealistic, Spectacular nature landscape, highly detailed, photo-postcard of the coast, including mountain, landscape, river, waterfall "},
                {"role": "assistant", "content":"Spectacular nature landscape, background mountains, river with waterfall, sharp focus, hyper detailed, detail colors, haze, fog, pine trees, depth, soft contrast,  golden hour colors, photo realistic, ultra realistic, cinematic landscape photography, cinematic ligthining"},
                {"role": "user", "content": 
             '''IDEA: ONE single image of photorealistic, tourism, nature landscape, ultra realistic, cinematic landscape photography,''' + input_text +"NO CANVAS"}
            ]
    )
    
    #print(response['choices'][0]['message']['content'])
    output = response['choices'][0]['message']['content']
    return output
    #os.system("say -v anna '"+output+"'") 
    #os.system("say '"+output+"'") 

def stability_generate(_number,input_prompt):
    # Set up our connection to the API.
    # select the engine using. (possible for stable diffusion 2)

    # Set up our initial generation parameters.
    answers = stability_api.generate(
        prompt=input_prompt,
        steps=40, # Amount of inference steps performed on image generation. Defaults to 30. 
        cfg_scale=8.0, # Influences how strongly your generation is guided to match your prompt.
                       # Setting this value higher increases the strength in which it tries to match your prompt.
                       # Defaults to 7.0 if not specified.
        width=1024, # Generation width, defaults to 512 if not included.
        height=768, # Generation height, defaults to 512 if not included.
    )

    IMAGE_DIR = Path.cwd() / "StableDiffusion_images"
    IMAGE_DIR.mkdir(parents=True, exist_ok=True)

    fileName = str(_number) + "image.png"
    for resp in answers:
        for artifact in resp.artifacts:
            if artifact.type == generation.ARTIFACT_IMAGE:
                image_file = IMAGE_DIR / fileName
                img = Image.open(io.BytesIO(artifact.binary))
                img.save(image_file) # Save our generated images with their seed number as the filename.



In [149]:
import csv
csv_file = "cluster_labels.csv"

# Save the labels to the CSV file
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Cluster", "Label"])
    writer.writerows(all_labels)
    
    
# Generate content and write in the same loop
for i in range(20):
    label = f"Cluster_{i}"
    #input_text = ','.join(cluster_label_lists[i])  # Adjust based on your actual cluster label data

    input_text = ','.join(cluster_label_lists[i])
    input_prompt = chatGPT(input_text)
    # Generate image
    
    print(str(i)+'\t'+input_prompt)
    #image_file = stability_generate(i, input_text)  # Adjust the function to return the filename/path

    # Generate advertisement
    prompt = f"A photorealistic, tourism-advertisement of {label}, ultra-realistic, cinematic photography, " + input_text
    advertisement = chatGPT_ads(prompt)

    # Write the output to CSV
    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow([label, image_file, advertisement])

    
    stability_generate(i,input_prompt)
    chatGPT_ads(input_text)


A breathtaking photorealistic portrayal of a tourism-worthy nature landscape, capturing the essence of the untamed wilderness. The scene showcases a majestic mountain range standing tall in the background, while a tranquil river meanders through the foreground, culminating in a stunning waterfall that cascades gracefully into the depths below. With impeccable attention to detail, every element of this masterpiece comes to life, from the vibrant colors of the flora and fauna to the misty haze that envelopes the scene. The lighting is carefully composed to mimic the golden hour, illuminating the landscape with a warm and cinematic glow. This ultra realistic depiction invites viewers to embark on a visual journey through the mesmerizing beauty of the northeastern, southern, southwest, northwest, western, northern, and even eastern regions, revealing the diverse and captivating landscapes that await exploration. Prepare to be transported to a world where nature reigns supreme in all its sp

KeyboardInterrupt: 

In [ ]:
import csv
import os
import time
csv_test = "csv_test.csv"

# Save the labels to the CSV file
with open(csv_test, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Cluster", "Label","imagePrompt","ads"])
 

    for i in range(20):
        input_label = ','.join(cluster_label_lists[i])
        input_prompt = chatGPT(input_label)
        
        #image_file = stability_generate(i, input_text)  # Adjust the function to return the filename/path

        # Generate advertisement
        prompt = f"A Really short tourism-advertisement with maximum 3 sentences containing the following text" + input_prompt
        advertisement = chatGPT_ads(prompt)
        writer.writerow([i, input_text, input_prompt, advertisement])
        time.sleep(1)


A breathtaking photorealistic tourism postcard capturing the beauty of a nature landscape in the northeastern, southern, southwestern, northwestern, western, and eastern regions. This ultra realistic image showcases a pristine coastline with majestic mountains, a winding river, and a stunning waterfall. The high level of detail brings the entire scene to life, with vibrant colors and soft contrast creating a cinematic atmosphere. The image is a true masterpiece of nature, offering a glimpse into the awe-inspiring beauty of various areas in the region. No canvas can contain the grandeur of this extraordinary landscape.
A photorealistic postcard features a spectacular nature landscape along the coastline. The scene captures the majestic mountains in the background, with a serene river flowing through. A waterfall cascades down, adding an element of both tranquility and dynamism. The attention to detail brings the entire scene to life, from the vibrant colors of the surroundings to the mi

In [145]:
for i in range(20):
    print(i,end=' ')
    print(','.join(cluster_label_lists[i]))


0 northeast,southern,southwest,northwest,western,northern,areas,region,eastern
1 boat,ship,vessel,fishing,boats,sail,sailing,ships,voyage,ashore,vessels,submarine,cruising,aboard,cruise,fleet,landing,sailed,collision,cargo,ferry,capsized,yacht
2 ocean,sea,waters,coast,atlantic,pacific,coastal,shores,shore,mediterranean,islands,island,inland,caribbean,harbor,gulf,cape,beach,archipelago,offshore,maritime,desert,harbour,peninsula,bay,hawaii,port
3 earth,surface,horizon,planet,discovery,mars,sky,space,outer,paradise,atmosphere,rocks,equator,journey
4 basin,lakes,lake,river,rivers,waterways,estuary,inlet,basins,freshwater,waterway
5 underwater,undersea,atmospheric,seismic,geological,depth,volcanic
6 seas,oceans,depths,oceanic,continental,continent
7 marine,rainforest,ecosystem,species,wildlife,habitat
8 arctic,antarctic,antarctica,seabed,alaska,polar,expedition,barents
9 waves,tidal,currents,tides,swells,tide
10 wind,weather,rain,icy,temperatures,dry,ice,warm
11 fish,whale,shark,whales
12 s